# Enhanced LLM-Powered Bag Recommendation System with Unsloth

In [ ]:
!pip install -q --upgrade pip
!pip install -q --upgrade unsloth
!pip install -q xformers trl peft accelerate bitsandbytes

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig
import json
import random
from typing import Dict, List, Tuple
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Create Dummy Bag Product Catalog

In [ ]:
class BagCatalog:
    def __init__(self):
        self.categories = ['Backpack', 'Messenger', 'Tote', 'Duffel', 'Sling', 'Laptop Bag', 'Travel', 'Crossbody']
        self.brands = ['NorthPeak', 'UrbanCarry', 'TechPack', 'Wanderlust', 'ProGear', 'EcoTote', 'SwissTrail', 'Minimalist']
        self.materials = ['Canvas', 'Nylon', 'Leather', 'Polyester', 'Recycled Plastic', 'Vegan Leather', 'Cotton', 'Ripstop']
        self.colors = ['Black', 'Navy', 'Gray', 'Brown', 'Olive', 'Burgundy', 'Tan', 'Charcoal']
        self.sizes = ['Small (15L)', 'Medium (25L)', 'Large (35L)', 'XL (45L+)']
        
        self.products = self.create_dummy_products()
    
    def create_dummy_products(self):
        products = [
            {'id': 0, 'name': 'NorthPeak Summit 30L Hiking Backpack', 'category': 'Backpack', 'brand': 'NorthPeak', 
             'price': 189.99, 'rating': 4.7, 'material': 'Ripstop', 'color': 'Olive', 'size': 'Large (35L)',
             'description': 'Rugged hiking backpack with hydration system, rain cover, and multiple compartments'},
            
            {'id': 1, 'name': 'UrbanCarry Metro Messenger', 'category': 'Messenger', 'brand': 'UrbanCarry',
             'price': 79.99, 'rating': 4.3, 'material': 'Canvas', 'color': 'Gray', 'size': 'Medium (25L)',
             'description': 'Urban messenger bag with padded laptop sleeve and quick-access pockets'},
            
            {'id': 2, 'name': 'TechPack Pro Laptop Backpack', 'category': 'Laptop Bag', 'brand': 'TechPack',
             'price': 129.99, 'rating': 4.8, 'material': 'Nylon', 'color': 'Black', 'size': 'Medium (25L)',
             'description': 'Tech-focused backpack with USB charging port, RFID protection, and cable management'},
            
            {'id': 3, 'name': 'EcoTote Sustainable Market Bag', 'category': 'Tote', 'brand': 'EcoTote',
             'price': 34.99, 'rating': 4.5, 'material': 'Recycled Plastic', 'color': 'Navy', 'size': 'Small (15L)',
             'description': 'Eco-friendly tote made from ocean plastic with reinforced handles'},
            
            {'id': 4, 'name': 'Wanderlust Expedition Duffel', 'category': 'Duffel', 'brand': 'Wanderlust',
             'price': 159.99, 'rating': 4.6, 'material': 'Polyester', 'color': 'Charcoal', 'size': 'XL (45L+)',
             'description': 'Large capacity duffel with shoe compartment and convertible backpack straps'},
            
            {'id': 5, 'name': 'ProGear Camera Sling Bag', 'category': 'Sling', 'brand': 'ProGear',
             'price': 89.99, 'rating': 4.4, 'material': 'Nylon', 'color': 'Black', 'size': 'Small (15L)',
             'description': 'Quick-access camera sling with customizable dividers and weather protection'},
            
            {'id': 6, 'name': 'SwissTrail Alpine Pack', 'category': 'Backpack', 'brand': 'SwissTrail',
             'price': 249.99, 'rating': 4.9, 'material': 'Ripstop', 'color': 'Brown', 'size': 'XL (45L+)',
             'description': 'Premium alpine backpack with advanced ventilation system and lifetime warranty'},
            
            {'id': 7, 'name': 'Minimalist Daily Crossbody', 'category': 'Crossbody', 'brand': 'Minimalist',
             'price': 59.99, 'rating': 4.2, 'material': 'Vegan Leather', 'color': 'Tan', 'size': 'Small (15L)',
             'description': 'Sleek crossbody bag with anti-theft zipper and adjustable strap'},
            
            {'id': 8, 'name': 'UrbanCarry Business Elite', 'category': 'Laptop Bag', 'brand': 'UrbanCarry',
             'price': 179.99, 'rating': 4.7, 'material': 'Leather', 'color': 'Brown', 'size': 'Medium (25L)',
             'description': 'Professional leather briefcase with laptop protection and executive styling'},
            
            {'id': 9, 'name': 'TechPack Student Edition', 'category': 'Backpack', 'brand': 'TechPack',
             'price': 69.99, 'rating': 4.1, 'material': 'Polyester', 'color': 'Navy', 'size': 'Medium (25L)',
             'description': 'Budget-friendly student backpack with tablet pocket and water bottle holders'},
        ]
        
        for i in range(10, 200):
            category = random.choice(self.categories)
            brand = random.choice(self.brands)
            material = random.choice(self.materials)
            color = random.choice(self.colors)
            size = random.choice(self.sizes)
            
            base_price = {'Backpack': 100, 'Messenger': 80, 'Tote': 40, 'Duffel': 120,
                         'Sling': 70, 'Laptop Bag': 110, 'Travel': 150, 'Crossbody': 60}[category]
            
            price = round(base_price + random.uniform(-30, 50), 2)
            rating = round(random.uniform(3.5, 5.0), 1)
            
            style_adj = random.choice(['Classic', 'Modern', 'Vintage', 'Sport', 'Professional', 'Casual'])
            
            products.append({
                'id': i,
                'name': f"{brand} {style_adj} {category}",
                'category': category,
                'brand': brand,
                'price': price,
                'rating': rating,
                'material': material,
                'color': color,
                'size': size,
                'description': f"{style_adj} {category.lower()} made from durable {material.lower()} in {color.lower()}. Features multiple compartments and comfortable straps."
            })
        
        return products
    
    def get_product(self, product_id):
        return self.products[product_id]
    
    def get_products_by_category(self, category):
        return [p for p in self.products if p['category'] == category]

catalog = BagCatalog()
print(f"Created bag catalog with {len(catalog.products)} products")
print(f"\nSample products:")
for product in catalog.products[:3]:
    print(f"  - {product['name']}: ${product['price']} ({product['material']}, {product['size']})")

In [ ]:
print("Detailed View of First 10 Dummy Products:")
print("="*80)
for i, product in enumerate(catalog.products[:10]):
    print(f"\n[Product #{product['id']}]")
    print(f"Name: {product['name']}")
    print(f"Category: {product['category']} | Brand: {product['brand']}")
    print(f"Material: {product['material']} | Color: {product['color']} | Size: {product['size']}")
    print(f"Price: ${product['price']} | Rating: {product['rating']}★")
    print(f"Description: {product['description']}")

print("\n" + "="*80)
print(f"Catalog Statistics:")
print(f"  Total Products: {len(catalog.products)}")
print(f"  Categories: {len(catalog.categories)}")
print(f"  Brands: {len(catalog.brands)}")
print(f"  Price Range: ${min(p['price'] for p in catalog.products):.2f} - ${max(p['price'] for p in catalog.products):.2f}")

In [ ]:
class BagUserProfileGenerator:
    def __init__(self, catalog):
        self.catalog = catalog
        self.user_profiles = self.create_dummy_users()
    
    def create_dummy_users(self):
        user_personas = [
            {'type': 'Student', 'preferred_categories': ['Backpack', 'Laptop Bag', 'Messenger'],
             'price_range': (30, 120), 'preferred_sizes': ['Medium (25L)', 'Small (15L)']},
            
            {'type': 'Business Professional', 'preferred_categories': ['Laptop Bag', 'Messenger', 'Crossbody'],
             'price_range': (100, 300), 'preferred_sizes': ['Medium (25L)']},
            
            {'type': 'Outdoor Enthusiast', 'preferred_categories': ['Backpack', 'Duffel', 'Sling'],
             'price_range': (80, 250), 'preferred_sizes': ['Large (35L)', 'XL (45L+)']},
            
            {'type': 'Fashion Conscious', 'preferred_categories': ['Tote', 'Crossbody', 'Messenger'],
             'price_range': (50, 200), 'preferred_sizes': ['Small (15L)', 'Medium (25L)']},
            
            {'type': 'Frequent Traveler', 'preferred_categories': ['Travel', 'Duffel', 'Backpack'],
             'price_range': (100, 350), 'preferred_sizes': ['Large (35L)', 'XL (45L+)']},
            
            {'type': 'Photographer', 'preferred_categories': ['Sling', 'Backpack', 'Messenger'],
             'price_range': (70, 250), 'preferred_sizes': ['Small (15L)', 'Medium (25L)']},
        ]
        
        dummy_purchase_histories = [
            [0, 2, 9],
            [1, 8, 5],
            [0, 4, 6],
            [3, 7, 1],
            [4, 6, 2],
            [5, 2, 0],
            [7, 3, 8],
            [9, 0, 1],
            [2, 8, 4],
            [6, 0, 5]
        ]
        
        users = []
        
        for user_id in range(10):
            persona = user_personas[user_id % len(user_personas)]
            users.append({
                'user_id': user_id,
                'type': persona['type'],
                'preferred_categories': persona['preferred_categories'],
                'price_range': persona['price_range'],
                'preferred_sizes': persona['preferred_sizes'],
                'purchase_history': dummy_purchase_histories[user_id]
            })
        
        for user_id in range(10, 100):
            persona = random.choice(user_personas)
            
            purchase_history = []
            for _ in range(random.randint(3, 10)):
                category = random.choice(persona['preferred_categories'])
                category_products = self.catalog.get_products_by_category(category)
                
                suitable_products = [
                    p for p in category_products 
                    if persona['price_range'][0] <= p['price'] <= persona['price_range'][1]
                ]
                
                if suitable_products:
                    product = random.choice(suitable_products)
                    purchase_history.append(product['id'])
            
            users.append({
                'user_id': user_id,
                'type': persona['type'],
                'preferred_categories': persona['preferred_categories'],
                'price_range': persona['price_range'],
                'preferred_sizes': persona['preferred_sizes'],
                'purchase_history': purchase_history
            })
        
        return users
    
    def get_user(self, user_id):
        return self.user_profiles[user_id]

users = BagUserProfileGenerator(catalog=catalog)
print(f"Generated {len(users.user_profiles)} user profiles")
print(f"\nSample user profiles:")
for user in users.user_profiles[:3]:
    print(f"  User {user['user_id']} ({user['type']}): {len(user['purchase_history'])} purchases")

In [ ]:
def generate_bag_training_data(users, catalog):
    training_data = [
        {'instruction': 'User previously bought: NorthPeak Summit 30L Hiking Backpack, TechPack Pro Laptop Backpack. Recommend a Travel product.',
         'response': 'I recommend: Wanderlust Expedition Duffel - Large capacity duffel with shoe compartment and convertible backpack straps',
         'user_id': 0, 'target_product_id': 4},
        
        {'instruction': 'User previously bought: UrbanCarry Metro Messenger, Minimalist Daily Crossbody. Recommend a Laptop Bag product.',
         'response': 'I recommend: UrbanCarry Business Elite - Professional leather briefcase with laptop protection and executive styling',
         'user_id': 1, 'target_product_id': 8},
        
        {'instruction': 'User previously bought: EcoTote Sustainable Market Bag, Minimalist Daily Crossbody. Recommend a Tote product.',
         'response': 'I recommend: EcoTote Canvas Shopping Tote - Eco-friendly tote made from organic canvas with reinforced handles',
         'user_id': 2, 'target_product_id': 3},
        
        {'instruction': 'A student needs a bag for college. They like tech features and moderate prices. What do you recommend?',
         'response': 'I recommend: TechPack Student Edition - Budget-friendly student backpack with tablet pocket and water bottle holders',
         'user_id': 3, 'target_product_id': 9},
        
        {'instruction': 'Looking for a professional bag for business travel with laptop protection under $200.',
         'response': 'I recommend: UrbanCarry Business Elite - Professional leather briefcase with laptop protection and executive styling',
         'user_id': 4, 'target_product_id': 8},
        
        {'instruction': 'User profile: Outdoor Enthusiast. Previous: hiking gear purchases. Need a durable pack for multi-day trips.',
         'response': 'I recommend: SwissTrail Alpine Pack - Premium alpine backpack with advanced ventilation system and lifetime warranty',
         'user_id': 5, 'target_product_id': 6},
        
        {'instruction': 'Customer bought ProGear Camera Sling Bag. Looking for a larger camera backpack with similar features.',
         'response': 'I recommend: TechPack Pro Laptop Backpack - Tech-focused backpack with USB charging port, RFID protection, and cable management',
         'user_id': 6, 'target_product_id': 2},
        
        {'instruction': 'Fashion-conscious user wants a stylish everyday bag in vegan leather under $100.',
         'response': 'I recommend: Minimalist Daily Crossbody - Sleek crossbody bag with anti-theft zipper and adjustable strap',
         'user_id': 7, 'target_product_id': 7},
        
        {'instruction': 'Previous purchases: small totes and crossbody bags. Customer prefers eco-friendly materials.',
         'response': 'I recommend: EcoTote Sustainable Market Bag - Eco-friendly tote made from ocean plastic with reinforced handles',
         'user_id': 8, 'target_product_id': 3},
        
        {'instruction': 'Frequent flyer needs expandable luggage with backpack straps for international travel.',
         'response': 'I recommend: Wanderlust Expedition Duffel - Large capacity duffel with shoe compartment and convertible backpack straps',
         'user_id': 9, 'target_product_id': 4}
    ]
    
    for _ in range(490):
        user = random.choice(users.user_profiles)
        
        if user['purchase_history']:
            context_products = random.sample(
                user['purchase_history'], 
                min(2, len(user['purchase_history']))
            )
            
            context_descriptions = [catalog.get_product(pid)['name'] for pid in context_products]
            
            target_category = random.choice(user['preferred_categories'])
            target_products = [
                p for p in catalog.get_products_by_category(target_category)
                if user['price_range'][0] <= p['price'] <= user['price_range'][1]
            ]
            
            if target_products:
                target_product = random.choice(target_products)
                
                prompt_templates = [
                    f"User previously bought: {', '.join(context_descriptions)}. Recommend a {target_category} product.",
                    f"Customer profile: {user['type']}. Looking for a {target_category} in the ${int(user['price_range'][0])}-${int(user['price_range'][1])} range.",
                    f"Based on purchase history of {context_descriptions[0]}, suggest a {target_category}.",
                    f"User likes {', '.join(user['preferred_categories'][:2])} bags. What {target_category} would you recommend?"
                ]
                
                prompt = random.choice(prompt_templates)
                response = f"I recommend: {target_product['name']} - {target_product['description']}"
                
                training_data.append({
                    'instruction': prompt,
                    'response': response,
                    'user_id': user['user_id'],
                    'target_product_id': target_product['id']
                })
    
    return training_data

training_data = generate_bag_training_data(users, catalog)
print(f"Generated {len(training_data)} training samples")
print(f"\nExample training samples:")
for i in range(3):
    print(f"\n{i+1}. {training_data[i]['instruction']}")
    print(f"   {training_data[i]['response']}")

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/tinyllama-bnb-4bit",
    max_seq_length=512,
    dtype=torch.float16,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
    max_seq_length=512,
)

print(f"Model loaded and LoRA applied")

In [ ]:
class RecommendationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        text = f"### Instruction: {item['instruction']}\n### Response: {item['response']}"
        
        encoded = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        
        return {
            'input_ids': encoded['input_ids'].squeeze(),
            'attention_mask': encoded['attention_mask'].squeeze(),
            'labels': encoded['input_ids'].squeeze()
        }

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=training_data[:400],
    eval_dataset=training_data[400:],
    dataset_text_field="instruction",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="adamw_8bit"
    ),
)

trainer.train()
print("Training complete!")

In [ ]:
class HybridRecommender(nn.Module):
    def __init__(self, model, tokenizer, embedding_dim=768, hidden_dim=256):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        
        self.user_encoder = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, embedding_dim)
        )
        
        self.product_encoder = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, embedding_dim)
        )
        
        self.fusion_layer = nn.Sequential(
            nn.Linear(embedding_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    
    def get_text_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.base_model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)
        
        return embeddings
    
    def recommend(self, user_history, candidate_products, top_k=5):
        user_text = f"User purchase history: {', '.join(user_history)}"
        user_embedding = self.get_text_embedding(user_text)
        user_features = self.user_encoder(user_embedding)
        
        scores = []
        for product in candidate_products:
            product_text = f"{product['name']} - {product['description']}"
            product_embedding = self.get_text_embedding(product_text)
            product_features = self.product_encoder(product_embedding)
            
            combined = torch.cat([user_features, product_features], dim=-1)
            score = self.fusion_layer(combined)
            scores.append((score.item(), product))
        
        scores.sort(key=lambda x: x[0], reverse=True)
        return scores[:top_k]

recommender = HybridRecommender(model, tokenizer).to(device)
print("Hybrid recommender initialized")

In [ ]:
def evaluate_recommendations(recommender, users, catalog, num_tests=10):
    results = []
    
    for _ in range(num_tests):
        user = random.choice(users.user_profiles)
        
        if len(user['purchase_history']) < 3:
            continue
        
        history_products = [catalog.get_product(pid)['name'] for pid in user['purchase_history'][:3]]
        
        candidate_products = random.sample(catalog.products, 20)
        
        recommendations = recommender.recommend(history_products, candidate_products, top_k=5)
        
        results.append({
            'user_id': user['user_id'],
            'history': history_products[:3],
            'recommendations': [(score, prod['name']) for score, prod in recommendations]
        })
    
    return results

evaluation_results = evaluate_recommendations(recommender, users, catalog, num_tests=5)

for i, result in enumerate(evaluation_results[:2]):
    print(f"\nUser {result['user_id']}:")
    print(f"History: {result['history'][:2]}")
    print(f"Top recommendations:")
    for score, name in result['recommendations'][:3]:
        print(f"  - {name} (score: {score:.3f})")

In [ ]:
class SessionAwareRecommender:
    def __init__(self, model, tokenizer, catalog):
        self.model = model
        self.tokenizer = tokenizer
        self.catalog = catalog
        self.session_embeddings = {}
    
    def update_session(self, session_id, viewed_products):
        product_names = [self.catalog.get_product(pid)['name'] for pid in viewed_products]
        session_text = f"Session products: {', '.join(product_names)}"
        
        inputs = self.tokenizer(session_text, return_tensors="pt", truncation=True)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.base_model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1)
        
        self.session_embeddings[session_id] = embedding
    
    def get_real_time_recommendations(self, session_id, candidate_product_ids, top_k=5):
        if session_id not in self.session_embeddings:
            return []
        
        session_emb = self.session_embeddings[session_id]
        
        scores = []
        for pid in candidate_product_ids:
            product = self.catalog.get_product(pid)
            product_text = f"{product['name']} - {product['category']}"
            
            inputs = self.tokenizer(product_text, return_tensors="pt", truncation=True)
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model.base_model(**inputs)
                product_emb = outputs.last_hidden_state.mean(dim=1)
            
            similarity = F.cosine_similarity(session_emb, product_emb)
            scores.append((similarity.item(), product))
        
        scores.sort(key=lambda x: x[0], reverse=True)
        return scores[:top_k]

session_recommender = SessionAwareRecommender(model, tokenizer, catalog)

session_id = "session_001"
viewed_products = [10, 25, 42]
session_recommender.update_session(session_id, viewed_products)

candidates = random.sample(range(len(catalog.products)), 30)
recommendations = session_recommender.get_real_time_recommendations(session_id, candidates, top_k=5)

print(f"Session {session_id} - Viewed: {[catalog.get_product(p)['name'] for p in viewed_products]}")
print(f"\nRecommendations:")
for score, product in recommendations:
    print(f"  {product['name']} - Score: {score:.3f}")

In [ ]:
def benchmark_performance():
    import time
    
    batch_sizes = [1, 4, 8, 16]
    results = {}
    
    for batch_size in batch_sizes:
        user_histories = []
        for _ in range(batch_size):
            user = random.choice(users.user_profiles)
            history = [catalog.get_product(pid)['name'] for pid in user['purchase_history'][:3]]
            user_histories.append(history)
        
        candidates = random.sample(catalog.products, 50)
        
        start_time = time.time()
        
        for history in user_histories:
            _ = recommender.recommend(history, candidates, top_k=5)
        
        end_time = time.time()
        
        avg_time = (end_time - start_time) / batch_size
        results[batch_size] = avg_time
        
        print(f"Batch size {batch_size}: {avg_time:.3f}s per user")
    
    return results

performance_results = benchmark_performance()

In [ ]:
def calculate_metrics(recommender, test_users, catalog, k=5):
    precisions = []
    recalls = []
    
    for user in test_users[:20]:
        if len(user['purchase_history']) < 5:
            continue
        
        train_history = user['purchase_history'][:3]
        test_items = user['purchase_history'][3:5]
        
        history_names = [catalog.get_product(pid)['name'] for pid in train_history]
        
        all_products = catalog.products[:100]
        recommendations = recommender.recommend(history_names, all_products, top_k=k)
        recommended_ids = [prod['id'] for _, prod in recommendations]
        
        hits = len(set(recommended_ids) & set(test_items))
        
        precision = hits / k if k > 0 else 0
        recall = hits / len(test_items) if test_items else 0
        
        precisions.append(precision)
        recalls.append(recall)
    
    avg_precision = np.mean(precisions) if precisions else 0
    avg_recall = np.mean(recalls) if recalls else 0
    f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0
    
    return {
        'precision@k': avg_precision,
        'recall@k': avg_recall,
        'f1_score': f1
    }

metrics = calculate_metrics(recommender, users.user_profiles, catalog, k=5)
print(f"\nEvaluation Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
def interactive_bag_demo():
    print("=" * 60)
    print("Interactive Bag Recommendation Demo")
    print("=" * 60)
    
    test_user = users.user_profiles[0]
    user_history = test_user['purchase_history'][:5]
    history_products = [catalog.get_product(pid) for pid in user_history]
    
    print(f"\nUser Profile:")
    print(f"User Type: {test_user['type']}")
    print(f"Preferred Bag Types: {', '.join(test_user['preferred_categories'])}")
    print(f"Budget Range: ${test_user['price_range'][0]:.2f} - ${test_user['price_range'][1]:.2f}")
    print(f"Preferred Sizes: {', '.join(test_user['preferred_sizes'])}")
    
    print(f"\nRecent Purchase History:")
    for product in history_products:
        print(f"  - {product['name']}")
        print(f"    {product['material']} | {product['color']} | {product['size']} | ${product['price']} | {product['rating']}★")
    
    candidate_products = [p for p in catalog.products[10:60] 
                         if test_user['price_range'][0] <= p['price'] <= test_user['price_range'][1]]
    
    history_names = [p['name'] for p in history_products]
    
    recommendations = recommender.recommend(history_names, candidate_products, top_k=10)
    
    print(f"\n{'='*60}")
    print(f"Top 10 Personalized Bag Recommendations for {test_user['type']}:")
    print(f"{'='*60}")
    
    for i, (score, product) in enumerate(recommendations, 1):
        match_cat = '✓' if product['category'] in test_user['preferred_categories'] else ' '
        match_size = '✓' if product['size'] in test_user['preferred_sizes'] else ' '
        
        print(f"\n{i:2}. {product['name']}")
        print(f"    Category Match [{match_cat}] | Size Match [{match_size}]")
        print(f"    {product['brand']} | {product['category']} | {product['material']} | {product['color']}")
        print(f"    Size: {product['size']} | Price: ${product['price']} | Rating: {product['rating']}★")
        print(f"    Recommendation Score: {score:.3f}")
        print(f"    {product['description']}")
    
    print(f"\n{'='*60}")
    print("Analysis Summary:")
    recommended_categories = [r[1]['category'] for r in recommendations]
    category_match_rate = sum(1 for cat in recommended_categories if cat in test_user['preferred_categories']) / len(recommendations) * 100
    
    avg_price = sum(r[1]['price'] for r in recommendations) / len(recommendations)
    in_budget = sum(1 for r in recommendations if test_user['price_range'][0] <= r[1]['price'] <= test_user['price_range'][1])
    
    print(f"  Category Match Rate: {category_match_rate:.1f}%")
    print(f"  Within Budget: {in_budget}/{len(recommendations)} recommendations")
    print(f"  Average Recommended Price: ${avg_price:.2f}")

interactive_bag_demo()